In [ ]:
# بخش اول: مدلسازی پرسپترون صرفا با numpy

In [ ]:
# perceptron.py
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

def load_and_prepare_numpy(csv_path):
    """
    Load the house price dataset, standardize features and
    convert continuous price to binary labels (-1, +1).
    """
    df = pd.read_csv(csv_path)
    # select feature columns
    feature_cols = ['Square_Footage', 'Num_Bedrooms', 'Num_Bathrooms',
                    'Year_Built', 'Lot_Size', 'Garage_Size', 'Neighborhood_Quality']
    X = df[feature_cols].values.astype(np.float32)
    # standardize features (zero mean, unit variance)
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    # convert price to binary labels: above-mean => +1, else -1
    prices = df['House_Price'].values
    mean_price = prices.mean()
    y = np.where(prices > mean_price, 1, -1).astype(np.int32)
    # split into train and test
    return train_test_split(X, y, test_size=0.3, random_state=42)

def perceptron_train_numpy(X_train, y_train, lr=0.01, epochs=50):
    """
    Train a perceptron using the classic update rule.
    Returns weight vector w and bias b.
    """
    n_features = X_train.shape[1]
    # initialize weights and bias to zero
    w = np.zeros(n_features, dtype=np.float32)
    b = 0.0
    # training loop
    for epoch in range(epochs):
        for xi, yi in zip(X_train, y_train):
            # compute the linear output
            activation = np.dot(w, xi) + b
            # perceptron prediction (+1 or -1)
            y_pred = 1 if activation >= 0 else -1
            # update only if misclassified
            if yi * activation <= 0:
                w += lr * yi * xi
                b += lr * yi
    return w, b

def perceptron_predict_numpy(X, w, b):
    """
    Predict class labels for X using learned w and b.
    """
    activations = X.dot(w) + b
    return np.where(activations >= 0, 1, -1)

if __name__ == "__main__":
    # load data
    X_train, X_test, y_train, y_test = load_and_prepare_numpy('house_price_regression_dataset.csv')
    # train
    w, b = perceptron_train_numpy(X_train, y_train, lr=0.01, epochs=100)
    # predict and evaluate
    y_pred = perceptron_predict_numpy(X_test, w, b)
    acc = np.mean(y_pred == y_test)
    print(f"Numpy Perceptron accuracy: {acc:.4f}")

In [ ]:
# 2) پیاده‌سازی پرسپترون با sklearn  

In [12]:
import pandas as pd
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def load_and_prepare_sklearn(csv_path):
    """
    Load dataset, split into train/test, and standardize features.
    """
    df = pd.read_csv(csv_path)
    feature_cols = ['Square_Footage', 'Num_Bedrooms', 'Num_Bathrooms',
                    'Year_Built', 'Lot_Size', 'Garage_Size', 'Neighborhood_Quality']
    X = df[feature_cols].values
    prices = df['House_Price'].values
    # binary labels: price above mean => +1, else -1
    y = (prices > prices.mean()).astype(int) * 2 - 1  # map to -1, +1
    # split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42)
    # standardize
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test, y_train, y_test

def run_sklearn_perceptron(X_train, y_train, X_test, y_test):
    """
    Train sklearn's Perceptron and evaluate accuracy.
    """
    clf = Perceptron(max_iter=1000, eta0=0.01, tol=1e-3, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"sklearn Perceptron accuracy: {acc:.4f}")

if __name__ == "__main__":
    X_train, X_test, y_train, y_test = load_and_prepare_sklearn('house_price_regression_dataset.csv')
    run_sklearn_perceptron(X_train, y_train, X_test, y_test)

sklearn Perceptron accuracy: 0.9733


In [ ]:
# 3) پیاده‌سازی پرسپترون با PyTorch  

In [13]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

def load_and_prepare_torch(csv_path, device='cpu'):
    """
    Load dataset, standardize features, convert to torch tensors.
    """
    df = pd.read_csv(csv_path)
    feature_cols = ['Square_Footage', 'Num_Bedrooms', 'Num_Bathrooms',
                    'Year_Built', 'Lot_Size', 'Garage_Size', 'Neighborhood_Quality']
    X = df[feature_cols].values.astype('float32')
    # standardize in numpy then convert
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    prices = df['House_Price'].values
    y = np.where(prices > prices.mean(), 1, -1).astype('int64')
    # split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42)
    # convert to torch
    X_train = torch.from_numpy(X_train).to(device)
    X_test = torch.from_numpy(X_test).to(device)
    y_train = torch.from_numpy(y_train).to(device)
    y_test = torch.from_numpy(y_test).to(device)
    return X_train, X_test, y_train, y_test

def perceptron_train_torch(X_train, y_train, lr=0.01, epochs=10):
    """
    Train perceptron manually using torch tensors.
    Returns weight vector w and bias b.
    """
    n_features = X_train.shape[1]
    # initialize parameters
    w = torch.zeros(n_features, dtype=torch.float32)
    b = torch.zeros(1, dtype=torch.float32)
    for epoch in range(epochs):
        for xi, yi in zip(X_train, y_train):
            # linear output
            output = torch.dot(w, xi) + b
            # prediction using sign
            y_pred = 1 if output.item() >= 0 else -1
            # update if wrong
            if yi.item() * output.item() <= 0:
                update = lr * yi.item()
                w += update * xi
                b += update
    return w, b

def perceptron_predict_torch(X, w, b):
    """
    Predict class labels for X using learned w and b.
    """
    outputs = X.matmul(w) + b
    return torch.where(outputs >= 0, 1, -1)

if __name__ == "__main__":
    import numpy as np
    # load data
    X_train, X_test, y_train, y_test = load_and_prepare_torch(
        'house_price_regression_dataset.csv', device='cpu')
    # train
    w, b = perceptron_train_torch(X_train, y_train, lr=0.01, epochs=50)
    # predict and evaluate
    y_pred = perceptron_predict_torch(X_test, w, b)
    acc = (y_pred == y_test).float().mean().item()
    print(f"PyTorch Perceptron accuracy: {acc:.4f}")

OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "C:\Users\Meysam\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\lib\c10.dll" or one of its dependencies.